<a href="https://colab.research.google.com/github/pstorniolo/Master2021/blob/main/2021_10_19_Intro_MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Intro-MapReduce

## Sommario

1. Richiami di *functional programming* in Python
2. Python `map` e `reduce` functions
3. Scrittura di codice parallelo con l'uso di `map`
4. Il modello completo di programmazione Map-Reduce



## Programmazione Funzionale

Consideriamo il seguente codice:

In [ ]:
def double_everything_in(data):
    result = []
    for i in data:
        result.append(2 * i)
    return result

def quadruple_everything_in(data):
    result = []
    for i in data:
        result.append(4 * i)
    return result

In [ ]:
double_everything_in([1, 2, 3, 4, 5])

In [ ]:
quadruple_everything_in([1, 2, 3, 4, 5])

### DRY (do not repeat yourself) - Concetto fondamentale nella programmazione

- Il codice precedente viola il principio ["non ripeterti"](https://en.wikipedia.org/wiki/Don't_repeat_yourself_) della buona pratica di ingegneria del software.

- Come si può riscrivere il codice in modo che eviti la duplicazione?

In [ ]:
def multiply_by_x_everything_in(x, data):
    result = []
    for i in data:
        result.append(x * i)
    return result

In [ ]:
multiply_by_x_everything_in(2, [1, 2, 3, 4, 5])

In [ ]:
multiply_by_x_everything_in(4, [1, 2, 3, 4, 5])

- Adesso consideriamo il seguente codice:

In [ ]:
def squared(x):
    return x*x

def double(x):
    return x*2

def square_everything_in(data):
    result = []
    for i in data:
        result.append(squared(i))
    return result

def double_everything_in(data):
    result = []
    for i in data:
        result.append(double(i))
    return result

In [ ]:
square_everything_in([1, 2, 3, 4, 5])

In [ ]:
double_everything_in([1, 2, 3, 4, 5])

### DRY (do not repeat yourself) - Concetto fondamentale nella programmazione

- Il codice precedente viola il principio ["non ripeterti"](https://en.wikipedia.org/wiki/Don't_repeat_yourself_) della buona pratica di ingegneria del software.

- Come si può riscrivere il codice in modo che eviti la duplicazione?

### Passaggio di valori ad una funzione
- Le funzioni possono essere passate ad altre funzioni come valori.


In [ ]:
def apply_f_to_everything_in(f, data):
    result = []
    for x in data:
        result.append(f(x))
    return result

In [ ]:
apply_f_to_everything_in(squared, [1, 2, 3, 4, 5])

In [ ]:
apply_f_to_everything_in(double, [1, 2, 3, 4, 5])

### Espressioni Lambda

- Possiamo usare funzioni anonime per evitare di dover definire una funzione ogni volta che vogliamo usare map.

In [ ]:
apply_f_to_everything_in(lambda x: x*x, [1, 2, 3, 4, 5])

## La funzione `map`

- Python ha una funzione integrata `map` che è molto più veloce della nostra versione.



In [ ]:
map(lambda x: x*x, [1, 2, 3, 4, 5])

## Implementazione di *reduce*

- La funzione `reduce` è un esempio di [fold](https://en.wikipedia.org/wiki/Fold_%28higher-order_function%29) *(reduce, accumulate, aggregate, compress, or inject)*.

- Ci sono diversi modi in cui possiamo effettuare il *fold* sui dati.

- Quanto segue implementa un *fold a sinistra*.


In [ ]:
def foldl(f, data, z):
    if (len(data) == 0):
        print (z)
        return z
    else:
        head = data[0]
        tail = data[1:]
        print ("Folding", head, "with", tail, "using", z)
        partial_result = f(z, data[0])
        print ("Partial result is", partial_result)
        return foldl(f, tail, partial_result)  

In [ ]:
def add(x, y):
    return x + y

foldl(add, [3, 3, 3, 3, 3], 0)

In [ ]:
foldl(lambda x, y: x + y, [1, 2, 3, 4, 5], 0)

In [ ]:
foldl(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)

In [ ]:
(((((0 - 1) - 2) - 3) - 4) - 5)

- La sottrazione non è né [commutativa](https://en.wikipedia.org/wiki/Commutative_property) né [associativa](https://en.wikipedia.org/wiki/Associative_property), quindi l'ordine in cui si applica il *fold* è determinante:

In [ ]:
(1 - (2 - (3 - (4 - (5 - 0)))))

In [ ]:
def foldr(f, data, z):
    if (len(data) == 0):
        return z
    else:
        return f(data[0], foldr(f, data[1:], z))                

In [ ]:
foldl(lambda x, y: x - y,  [1, 2, 3, 4, 5], 0)

In [ ]:
foldr(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)

## La funzione `reduce` di Python.

- La funzione `reduce` incorporata in Python è una *fold sinistra*.

In [ ]:
from functools import reduce
reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])

In [ ]:
reduce(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)

In [ ]:
foldl(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)

## Programmazione funzionale e parallelismo

- La programmazione funzionale si presta alla [programmazione parallela](https://computing.llnl.gov/tutorials/parallel_comp/#Models).

- La funzione `map` può essere facilmente parallelizzata tramite [parallelismo a livello di dati](https://en.wikipedia.org/wiki/Data_parallelism),
     - a condizione che la funzione che forniamo come argomento sia *libera da* [effetti collaterali](https://en.wikipedia.org/wiki/Side_effect_%28computer_science%29)
         - (ecco perché evitiamo di lavorare con dati mutevoli).

- Possiamo vederlo riscrivendo il codice così:

In [ ]:
def perform_computation(f, result, data, i):
    print ("Computing the ", i, "th result...")
    # This could be scheduled on a different CPU
    result[i] = f(data[i])

def my_map(f, data):
    result = [None] * len(data)
    for i in range(len(data)):
        perform_computation(f, result, data, i)
    # Wait for other CPUs to finish, and then..
    return result

In [ ]:
my_map(lambda x: x * x, [1, 2, 3, 4, 5])

## Una funzione `map` multi-thread

In [ ]:
from threading import Thread

def schedule_computation_threaded(f, result, data, threads, i):    
    # Each function evaluation is scheduled on a different core.
    def my_job(): 
        print ("Processing data:", data[i], "... ")
        result[i] = f(data[i])
        print ("Finished job #", i)    
        print ("Result was", result[i])       
    threads[i] = Thread(target=my_job)
    
def my_map_multithreaded(f, data):
    n = len(data)
    result = [None] * n
    threads = [None] * n
    print ("Scheduling jobs.. ")
    for i in range(n):
        schedule_computation_threaded(f, result, data, threads, i)
    print ("Starting jobs.. ")
    for i in range(n):
        threads[i].start()
    print ("Waiting for jobs to finish.. ")
    for i in range(n):
        threads[i].join()
    print ("All done.")
    return result

In [ ]:
my_map_multithreaded(lambda x: x*x, [1, 2, 3, 4, 5])

In [ ]:
from numpy.random import uniform
from time import sleep

def a_function_which_takes_a_long_time(x):
    sleep(uniform(2, 10))  # Simulate some long computation
    return x*x

my_map_multithreaded(a_function_which_takes_a_long_time, [1, 2, 3, 4, 5])

## Map Reduce

- Map Reduce è un _modello di programmazione_ per l'elaborazione parallela scalabile.
- Scalabile qui significa che può funzionare su big data con cluster di calcolo molto grandi.
- Ci sono molte implementazioni: ad es. Apache Hadoop e Apache Spark.
- Possiamo utilizzare Map-Reduce con qualsiasi linguaggio di programmazione:
     - Hadoop è scritto in Java
     - Spark è scritto in Scala, ma ha un'interfaccia Python.
- Linguaggi di *programmazione funzionale* come Python o Scala si adattano molto bene al modello Map Reduce:
     - Tuttavia, non *dobbiamo* usare la programmazione funzionale.

- Un'implementazione di MapReduce si occuperà delle funzionalità di basso livello in modo che non ci si debba preoccupare di:
     - bilancio del carico
     - I/O di rete
     - ottimizzazione del trasferimento di rete e su disco
     - gestione guasti macchina
     - serializzazione dei dati
     - eccetera..
- Il modello è progettato per spostare l'elaborazione nel luogo in cui risiedono i dati.

## Passaggi tipici in un calcolo Map-Reduce

1. ETL un grande set di dati.
2. Operazione _Map_: estrai qualcosa che ti interessa da ogni riga
3. "Shuffle and Sort": allocazione task/nodo
4. Operazione _Riduci_: aggrega, riepiloga, filtra o trasforma
5. Scrivi i risultati.

## Callback per Map Reduce

- Il set di dati e lo stato di ogni fase del calcolo è rappresentato come un insieme di coppie chiave-valore.

- Il programmatore fornisce una funzione mappa:

  $\operatorname{map}(k, v) \rightarrow \; \left< k', v' \right>*$

- e una funzione di riduzione:

  $\operatorname{reduce}(k', \left< k', v'\right> *) \rightarrow \; \left< k', v''
\right> *$

- Indicando con $*$ una *raccolta* di valori.

- Queste raccolte *non* sono ordinate.

## Esempio di conteggio parole

- In questo semplice esempio, l'input è un insieme di URL, ogni record è un documento.

- Problema: calcola quante volte ogni parola si è presentata nel set di dati.

## Conteggio parole: mappa


- L'input di $\operatorname{map}$ è una mappatura:

  - Chiave: URL
  - Valore: contenuto del documento

$\left< document1, to \; be \; or \; not \; to \; be \right>$      

- In questo esempio, la nostra funzione $\operatorname{map}$ elaborerà un determinato URL e produrrà una mappatura:

- Key: parola
- Value: 1

- Quindi il nostro set di dati originale sarà trasformato in:
  
  $\left< to, 1 \right>$
  $\left< be, 1 \right>$
  $\left< or, 1 \right>$
  $\left< not, 1 \right>$
  $\left< to, 1 \right>$
  $\left< be, 1 \right>$

## Conteggio parole: Reduce


- L'operazione di riduzione raggruppa i valori in base alla loro chiave, quindi esegue l'operazione di riduzione su ogni chiave.

- Le collezioni sono quindi suddivise in diverse unità di archiviazione.

- Map-Reduce effettuerà il *fold* dei dati in modo tale da ridurre al minimo la copia dei dati attraverso il cluster.

- I dati in diverse partizioni vengono ridotti separatamente in parallelo.

- Il risultato finale è una *reduce* dei dati in ogni partizione.

- Quindi è molto importante che il nostro operatore essere *sia commutativo che associativo*.

- Nel nostro caso la funzione è l'operatore `+`

  $\left< be, 2 \right>$  
  $\left< not, 1 \right>$  
  $\left< or, 1 \right>$  
  $\left< to, 2 \right>$  
  

## Map e reduce rispetto a Python

- Notare che queste funzioni sono formulate in modo diverso dalle funzioni standard di Python con lo stesso nome.

- La funzione `reduce` funziona con *coppie* di valori-chiave.

- Sarebbe più appropriato chiamarlo qualcosa come `reduceByKey`.

## MiniMapReduce

- Per illustrare come funziona il modello di programmazione Map-Reduce, possiamo implementare il nostro framework Map-Reduce in Python.

- Questo *illustra* come un problema può essere scritto in termini di operazioni `map` e `reduce`.

- Notare che queste sono funzioni illustrative; questo *non* è il modo in cui Hadoop o Apache Spark li implementano effettivamente.

In [ ]:
##########################################################
#
#   MiniMapReduce
#
# A non-parallel, non-scalable Map-Reduce implementation
##########################################################

def groupByKey(data):
    result = dict()
    for key, value in data:
        if key in result:
            result[key].append(value)
        else:
            result[key] = [value]
    return result
        
def reduceByKey(f, key_values):
    return map(lambda key: (key, reduce(f, key_values[key])), key_values.keys())

## Conteggio parole utilizzando MiniMapReduce

In [ ]:
data = map(lambda x: (x, 1), "to be or not to be".split())

In [ ]:
key_values = groupByKey(data)
print(key_values)

In [ ]:
dict(reduceByKey(lambda x, y: x + y, key_values))


## Parallelizzare MiniMapReduce

- Possiamo trasformare la nostra implementazione di Map-Reduce in un framework parallelo e multi-thread
utilizzando la funzione `my_map_multithreaded` definita in precedenza.

- Questo ci consentirà di eseguire calcoli di riduzione della mappa che sfruttano l'elaborazione parallela utilizzando *più* core su un *singolo* computer.

In [ ]:
def reduceByKey_multithreaded(f, data):
    key_values = groupByKey(data)
    return my_map_multithreaded(lambda key: (key, reduce(f, key_values[key])), key_values.keys())

In [ ]:
reduceByKey_multithreaded(lambda x, y: x + y, data)

## Parallelizzazione della fase di riduzione

- A condizione che il nostro operatore sia associativo e commutativo possiamo parallelizzare anche l'operazione di riduzione.

- Dividiamo i dati in sottoinsiemi approssimativamente uguali.

- Quindi riduciamo ogni sottoinsieme indipendentemente su un core separato.

- I risultati possono essere combinati in una fase di riduzione finale.

### Partizionamento dei dati

In [ ]:
def split_data(data, split_points):
    partitions = []
    n = 0
    for i in split_points:
        partitions.append(data[n:i])
        n = i
    partitions.append(data[n:])
    return partitions

data = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
partitioned_data = split_data(data, [3])
partitioned_data

### Riduzione tra le partizioni in parallelo

In [ ]:
from threading import Thread

def parallel_reduce(f, partitions):

    n = len(partitions)
    results = [None] * n
    threads = [None] * n
    
    def job(i):
        results[i] = reduce(f, partitions[i])

    for i in range(n):
        threads[i] = Thread(target = lambda: job(i))
        threads[i].start()
    
    for i in range(n):
        threads[i].join()
    
    return reduce(f, results)

parallel_reduce(lambda x, y: x + y, partitioned_data)

## Map-Reduce su un cluster di computer

- Il codice che abbiamo scritto finora *non* ci permetterà di sfruttare il parallelismo da più computer in un [cluster](https://en.wikipedia.org/wiki/Computer_cluster).

- Lo sviluppo di un tale framework sarebbe un progetto di ingegneria del software molto ampio.

- Esistono framework che possiamo utilizzare:
     - [Apache Hadoop](https://hadoop.apache.org/)
     - [Apache Spark](https://spark.apache.org/)